In [2]:
import pandas as pd
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import random
!pip install stackapi
from stackapi import StackAPI
unique_tags_dict_wiki = pd.read_pickle(r'/content/unique_tags_dict_wiki.pkl')
user_tags_dict = pd.read_pickle(r'/content/user_tags_dict.pkl')
userid = pd.read_pickle(r'/content/User_Id.pkl')
final_most_relevant_tag_old= pd.read_pickle(r'/content/final_most_relevant_tag.pkl')
relevant_tag=pd.read_pickle(r'/content/relavent_tag.pkl')
nltk.download('punkt')
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stackapi: filename=StackAPI-0.2.0-py3-none-any.whl size=5859 sha256=db4e5694413ec6654c56b595ec87a7160e783884f44944cee0fae9c4fb85b79c
  Stored in directory: /root/.cache/pip/wheels/15/38/46/ce92f4ec7e81b3da9b2bae72b572a49a2b1b025cdce325cc01
Successfully built stackapi


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# to get intersection of relavent tags from Chars2Vec and stackovwerflow data.
tags=list(unique_tags_dict_wiki.keys())
intersection_tags_count={}
def intersection_two_dict(x):
  a=final_most_relevant_tag_old[x]
  b=relevant_tag[x]
  return list(set(a) & set(b))
for tag in tags:
  intersection_tags_count[tag]=len(intersection_two_dict(tag))

# collecting all unique tags
tags=list(unique_tags_dict_wiki.keys())

# Prepaing empty one hot vector
all_user_v={}
for user in userid:
  all_user_v[user]=[None]*len(tags)

# this method is based on similarity calculation and relavent method from stackoverflow
def cal_vect(user,user_v):
  for i in range(len(tags)):
    if tags[i] in user:
      relavent_tags=relevant_tag[tags[i]]
      for k in range(len(tags)):
        for r_tag in relavent_tags:
          if r_tag==tags[k]:
            user_v[k]=1
      user_v[i]=1
    else:
      user_v[i]=0
  return user_v

# Prepaing one hot encoding vector
all_list=[]
for user in userid:
  all_list.append(cal_vect(user_tags_dict[user],all_user_v[user]))


# adding data to one hot encoder
import pandas as pd
df = pd.DataFrame(all_list, columns=tags)
data_items=df
magnitude = np.sqrt(np.square(data_items).sum(axis=1))
data_items = data_items.divide(magnitude, axis='index')

# cosine_similarity
def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim
data_matrix = calculate_similarity(data_items)

# include user index numnber
user_index=6
known_user_likes = data_items.iloc[user_index]
known_user_likes = known_user_likes[known_user_likes >0].index.values

user_rating_vector = data_items.iloc[user_index]
score = data_matrix.dot(user_rating_vector).div(data_matrix.sum(axis=1))
data_matrix.dot(user_rating_vector).div(data_matrix.sum(axis=1))
# user_rating_vector
score = score.drop(known_user_likes)
# Print the known likes and the top 20 recommendations.
# print(known_user_likes)
predicted_list=list(score.nlargest(20).index)


In [4]:
# validating the model
!pip install stackapi
SITE = StackAPI('stackoverflow',key='44hpaaJESWN4fN22RgRZTg((')
comments = SITE.fetch('questions/unanswered',tagged=[predicted_list])
length=len(comments['items'])
for l in range(length):
  print(comments['items'][l]['title'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
How can I modify my __repr__ to respresent correctly?
Bad string representation of negative imaginary numbers in Python
Why I am getting error by placing self in the __repr__ method in the class in python?
Why does Pycharm Console do this with my __repr__ method?
Nativecall Segfaults Getting Tuple from Rust
Why __repr__ and __str__ return a None value?
How can I get a repr of a DataFrame that is a valid Python expression?
python eval(repr(file_handle)) fails with synax error
How to write a __repr__ when one of the inputs is a dataframe?
Is there a way to search in an __repr__ similar to a string in Python
Why do I cannot use __repr__ data outside of a class
Cannot print object repr using __repr__ in Python
__init__ and create a linked list in Python
Repr of lxml node
In Python3 the behaviour of the inbuilt repr() function has changed when compared to Python2
Define different output for th